# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import normal, choice
from azureml.core.experiment import Experiment
from azureml.core.dataset import Dataset
import os
import shutil
import joblib

In [2]:
ws = Workspace.from_config()
experiment_name = 'online-shopping-intension'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location,  
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-136580
Azure region: southcentralus
Resource group: aml-quickstarts-136580


In [5]:
cpu_cluster_name = "project3"

try:
    cpu_cluster = ComputeTarget(workspace=ws,name=cpu_cluster_name)
    print("CPU Cluster exist")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,cpu_cluster_name,compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [6]:
found = False
key = "online_shoppers_intention"
description_text = "Online shopping intention of customer"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # link to the dataset
        example_data = 'https://raw.githubusercontent.com/Bhosalenileshn/ML-Azure-Udacity-Capstone-Project/main/online_shoppers_intention.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12316.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.317798,80.906176,0.503979,34.506387,31.763884,1196.037057,0.022152,0.043003,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.322754,176.860432,1.270701,140.825479,44.490339,1914.372511,0.048427,0.048527,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,185.000000,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,8.000000,0.000000,0.000000,18.000000,599.766190,0.003119,0.025124,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.500000,0.000000,0.000000,38.000000,1466.479902,0.016684,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


In [7]:

from azureml.core import Workspace, Dataset

# subscription_id = 'aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee'
# resource_group = 'aml-quickstarts-135086'
# workspace_name = 'quick-starts-ws-135086'

#workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace=ws, name='online_shoppers_intention')
dataset.to_pandas_dataframe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0.0,0.0,0.0,0.0,1.0,0.000000,0.200000,0.200000,0.000000,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0.0,0.0,0.0,0.0,2.0,64.000000,0.000000,0.100000,0.000000,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0.0,-1.0,0.0,-1.0,1.0,-1.000000,0.200000,0.200000,0.000000,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0.0,0.0,0.0,0.0,2.0,2.666667,0.050000,0.140000,0.000000,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0.0,0.0,0.0,0.0,10.0,627.500000,0.020000,0.050000,0.000000,0.0,Feb,3,3,1,4,Returning_Visitor,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3.0,145.0,0.0,0.0,53.0,1783.791667,0.007143,0.029031,12.241717,0.0,Dec,4,6,1,1,Returning_Visitor,True,False
12326,0.0,0.0,0.0,0.0,5.0,465.750000,0.000000,0.021333,0.000000,0.0,Nov,3,2,1,8,Returning_Visitor,True,False
12327,0.0,0.0,0.0,0.0,6.0,184.250000,0.083333,0.086667,0.000000,0.0,Nov,3,2,1,13,Returning_Visitor,True,False
12328,4.0,75.0,0.0,0.0,15.0,346.000000,0.000000,0.021053,0.000000,0.0,Nov,2,2,3,11,Returning_Visitor,False,False


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy.
  
We used scikit learn Logistic Regrssion algorithm for the predicion as this task is about Classification. We created train.py script to do that.
***train.py***  
This script accepts three arguments.  
   1. C : Inverse of regularization [For More Details](https://stackoverflow.com/questions/22851316/what-is-the-inverse-of-regularization-strength-in-logistic-regression-how-shoul)  
   2. max_iter : Maximum number of iterations taken for the solvers to converge. 
   
   3. penalty(regularization) :l1 and l2 regularization [For more Details](https://medium.com/analytics-vidhya/l1-vs-l2-regularization-which-is-better-d01068e6658c#:~:text=The%20main%20intuitive%20difference%20between,the%20data%20to%20avoid%20overfitting.&text=That%20value%20will%20also%20be%20the%20median%20of%20the%20data%20distribution%20mathematically.)
  
  
Logistic Regression algorithm is used from the scikit learn.

***Tuning the Hyperparameters***    
 If we tune the HyperParameters manually it will take log time. So, we used the Azure HyperDrive tool to find and tune the best Hyperparameters. We followed the below steps.
   
   1.  Used RandomParameterSampler as it is fast as compared to grid search and saves the budget to exhaustively search over the search space.  
       The parameter search spcae used for *C* and *max_iter* is choice(1,2,3,4,5) and choice(100,150,200,250) resp.  
       [For more Details on RandomSampling](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters)  
   2.  Specified the Early stopping policy as *BanditPolicy with slack_factor as 0.1, evaluation_interval as 1 and delay_evaluation as 5*   
       Bandit is an early termination policy based on slack factor/slack amount and evaluation interval.   
       The policy early terminates any runs where the primary metric is not within the specified slack factor/slack amount with respect to the best performing     training run.  
       [For more Details on Bandit Policy](https://azure.github.io/azureml-sdk-for-r/reference/bandit_policy.html)  

In [8]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
    {'--C' : choice(1,2,3,4,5),
     '--max_iter' : choice(100,150,200,250),
     '--penalty' : choice('l1', 'l2')}
)

script_folder = './training'
os.makedirs(script_folder, exist_ok=True)

shutil.copy('./train.py', script_folder)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=script_folder,compute_target=cpu_cluster,entry_script='train.py')
hyperdrive_run_config = HyperDriveConfig(estimator=estimator,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [9]:
#TODO: Submit your experiment
experiment = Experiment(workspace=ws, name='HyperDriveMLOptimize')
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '4', '--max_iter', '150', '--penalty', 'l2']


In [12]:
#TODO: Save the best model
model = best_run.register_model(model_name='classification_hyperdrive', model_path='outputs/model.joblib')

In [13]:
best_run_metrics = best_run.get_metrics()
# parameter_values = best_run.get_details()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_7dc14c5e-f3e2-454f-b361-f63ddd300269_4

 Accuracy: 0.878345498783455


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service